# Lab 6 - Data Warehouse Management

## 6.1 - Checking important warehouse settings

In [ ]:
--step 6.1.1
SELECT
    name,
    collation_name,
    CASE collation_name
        WHEN 'Latin1_General_100_BIN2_UTF8'           THEN 1
        WHEN 'Latin1_General_100_CI_AS_KS_WS_SC_UTF8' THEN 0
        ELSE NULL
        END AS is_case_sensitive,
    snapshot_isolation_state_desc,
    is_auto_create_stats_on,
    is_auto_update_stats_on,
    data_lake_log_publishing_desc,
    CASE data_lake_log_publishing_desc
        WHEN 'AUTO' THEN 1
        WHEN 'PAUSED' THEN 0
        ELSE NULL
        END AS is_delta_lake_log_publishing_on,
    is_vorder_enabled
FROM sys.databases
WHERE name = DB_NAME()

## 6.2 - Statistics management

In [ ]:
--step 6.2.1
SELECT
    schema_name(o.schema_id) AS schema_name,
    object_name(s.object_id) AS [object_name],
    c.name AS [column_name],
    s.name AS [stats_name],
    STATS_DATE(s.object_id, s.stats_id) AS [stats_update_date], 
    CASE
        WHEN s.auto_created = 1 THEN 'System'
        WHEN s.user_created = 1 THEN 'User'
        ELSE NULL END AS stats_created_by
FROM sys.stats AS s 
INNER JOIN sys.objects AS o 
    ON o.object_id = s.object_id 
LEFT JOIN sys.stats_columns AS sc 
    ON s.object_id = sc.object_id 
    AND s.stats_id = sc.stats_id 
LEFT JOIN sys.columns AS c 
    ON sc.object_id = c.object_id 
    AND c.column_id = sc.column_id
WHERE
    o.type = 'U'
    AND (s.auto_created = 1 or s.user_created = 1)
ORDER BY
    schema_name,
    object_name,
    column_name

In [ ]:
--step 6.2.2
CREATE STATISTICS dbo_DimBrand_BrandSK ON dbo.DimBrand (BrandSK) WITH FULLSCAN;
CREATE STATISTICS dbo_DimStockItem_StockItemSK ON dbo.DimStockItem (StockItemSK) WITH FULLSCAN;
CREATE STATISTICS dbo_DimCustomer_CustomerSK ON dbo.DimCustomer (CustomerSK) WITH FULLSCAN;

In [ ]:
--step 6.2.3
SELECT
    schema_name(o.schema_id) AS schema_name,
    object_name(s.object_id) AS [object_name],
    c.name AS [column_name],
    s.name AS [stats_name],
    STATS_DATE(s.object_id, s.stats_id) AS [stats_update_date], 
    CASE
        WHEN s.auto_created = 1 THEN 'System'
        WHEN s.user_created = 1 THEN 'User'
        ELSE NULL END AS stats_created_by
FROM sys.stats AS s 
INNER JOIN sys.objects AS o 
    ON o.object_id = s.object_id 
LEFT JOIN sys.stats_columns AS sc 
    ON s.object_id = sc.object_id 
    AND s.stats_id = sc.stats_id 
LEFT JOIN sys.columns AS c 
    ON sc.object_id = c.object_id 
    AND c.column_id = sc.column_id
WHERE
    o.type = 'U'
    AND (s.auto_created = 1 or s.user_created = 1)
ORDER BY
    schema_name,
    object_name,
    column_name

## Step 6.3 - Query monitoring

In [ ]:
--step 6.3.1
SELECT TOP 10 * 
FROM queryinsights.exec_requests_history 
WHERE
    submit_time > DATEADD(hour, -4, GETDATE()) 
    AND data_scanned_remote_storage_mb + data_scanned_memory_mb + data_scanned_disk_mb > 0
ORDER BY submit_time DESC